In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.linear_model import LogisticRegressionCV

In [2]:
data_dir = Path().cwd().parent / 'data'

In [6]:
print(data_dir)

c:\Users\hbjro\AppData\Local\Programs\data


In [5]:
la_region_lookup = pd.read_csv(data_dir/'la_region_lookup.csv')
la_urban_rural = pd.read_csv(data_dir/'la_urban_rural.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\hbjro\\AppData\\Local\\Programs\\data\\la_region_lookup.csv'

In [ ]:
def get_vote(x): 
    if (x == 'An independent candidate') | (x == "Don't know") | (x == 'Other') | (x == 'United Kingdom Independence Party (UKIP)') | (x == 'Plaid Cymru') | (pd.isna(x)) :
        return np.nan
    else: return x

def get_ethncity(x):

    if (x == 'Prefer not to say') | pd.isna(x):
        return np.nan
    elif (x == 'White British') | (x == 'Any other white background'):
        return 'White'
    elif (x == 'White and Asian') | (x == 'White and Black African') | (x == 'White and Black Caribbean') | (x == 'Any other mixed background'):
        return 'Mixed Race'
    elif (x == 'Pakistani') | (x == 'Indian') | (x == 'Chinese') | (x == 'Bangladeshi') | (x == 'Any other Asian background'):
        return 'Asian'
    elif (x == 'Black African') | (x == 'Black Caribbean') | (x == 'Any other black background'):
        return 'Black'
    else: return x

def get_religion(x):

    if (x == 'Yes - Roman Catholic'):
        return 'Catholic'
    elif (x == 'Yes - Baptist') | (x == 'Yes - Free Presbyterian') | (x == 'Yes - Brethren') | (x == 'Yes - United Reformed Church') | \
    (x == 'Yes - Methodist') | (x == 'Yes - Presbyterian/Church of Scotland') |\
    (x == 'Yes - Evangelical – independent/non-denominational (e.g. FIEC, Pioneer, Vineyard, Newfrontiers)') |\
    (x == 'Yes - Pentecostal (e.g. Assemblies of God, Elim Pentecostal Church, New Testament Church of God, Redeemed Christian Chur'):
        return 'non-Anglican Protestant'
    elif (x == 'Yes - Church of England/Anglican/Episcopal'): 
        return 'Anglican'
    elif (x == 'Yes - Islam'): 
        return 'Islam'
    elif (x == 'Yes - Judaism'):
        return 'Judaism'
    elif (x == 'No, I do not regard myself as belonging to any particular religion.'):
        return 'No religion'
    elif (x == 'Prefer not to say'):
        return np.nan
    else: return 'Other'

def get_sexuality(x):
    if (x == 'Bisexual') | (x == 'Gay or lesbian') | (x == 'Other'):
        return ('LGB+')
    elif (x == 'Heterosexual'):
        return ('Straight')
    elif (x == 'Prefer not to say'):
        return np.nan
    else: return x

def get_housing(x):
    if x == 'Own – outright': 
        return 'own outright' 
    elif x == 'Own – with a mortgage' or x == 'Own (part-own) – through shared ownership scheme (i.e. pay part mortgage, part rent)': 
        return 'own with mortgage'
    elif x == "Rent – from a private landlord": 
        return 'private rental' 
    elif x == 'Rent – from a housing association' or x == 'Rent – from my local authority':
        return 'social housing' 
    else: 
        return 'other'

def get_work(x):
    if (x == 'Working full time (30 or more hours per week)'):
        return 'full time'
    if (x == 'Working part time (8-29 hours a week)') | (x == 'Working part time (Less than 8 hours a week)'):
        return 'part time'
    if (x == 'Full time student'): 
        return 'student'
    if (x == 'Retired'):
        return 'retired'
    if (x == 'Unemployed'):
        return 'unemployed'
    else: 
        return 'Other'

def get_age_cat(x):
    if x < 18: 
        return np.nan 
    elif x < 30: 
        return '18-29' 
    elif x < 45: 
        return '30-44'
    elif x < 65: 
        return '45-65' 
    else:
        return '65+' 

def get_married(x):
    if (x == 'Married') | (x == 'In a civil partnership') | (x == 'Separated but still legally married or in a civil partnership'):
        return 'Married'
    elif (x == 'Divorced') | (x == 'Single') | (x == 'Widowed') |\
    (x =='In a relationship, but not living together') | (x == 'Living with a partner but neither married nor in a civil partnership'):
        return 'Unmarried'

def get_children(x):
    if (x == 'Yes'):
        return 'children'
    elif (x == 'No'): 
        return 'no children'

def get_income(x):
    if (x == "Don't know") | (x == 'Prefer not to answer'):
        return np.nan
    else: return x

def get_education(x):
    if (x == 'No formal qualifications'):
        return ("No qualifications")
    if (x == 'CSE grade 1, GCE O level, GCSE, School Certificate') | (x == 'CSE grades 2-5') | (x == 'Scottish Ordinary/ Lower Certificate'):
        return ('GCSE or equivalent')
    if (x == 'GCE A level or Higher Certificate') | (x == 'Scottish Higher Certificate') | (x == 'ONC'):
        return 'A-level or equivalent'
    if (x == 'University or CNAA first degree (e.g. BA, B.Sc, B.Ed)') | (x == 'University diploma'):
        return 'University degree'
    if (x == 'University or CNAA higher degree (e.g. M.Sc, Ph.D)'):
        return 'Post-graduate degree'
    if (x == "Don't know") | (x == 'Prefer not to say'):
        return np.nan
    if (x == 'Other technical, professional or higher qualification') | (x == 'City & Guilds certificate') |\
    (x == 'City & Guilds certificate - advanced') | (x == 'Teaching qualification (not degree)') | ( x == 'Clerical and commercial') |\
    (x == 'Nursing qualification (e.g. SEN, SRN, SCM, RGN)') | (x == 'Recognised trade apprenticeship completed') |\
    (x == 'Youth training certificate/skillseekers'):
        return 'Technical or professional qualification'
    else: return x


def get_region(x):
    if (x == 'Buckinghamshire'):
        return 'South East'
    if (x == 'West Northamptionshire'):
        return 'East Midlands'
    if (x == 'North Northamptonshire'):
        return 'East Midlands'
    else: 
        try: 
            return la_region_lookup[la_region_lookup['LAD19NM'] == x]['RGN19NM'].values[0]
        except: 
            return np.nan

def get_urban_rural(x):
    if x == 'Dorset': return 'rural'
    elif x == 'Buckinghamshire': return 'mostly rural'
    elif x == 'Bournemouth, Christchurch and Pool': return 'urban'
    elif x == 'West Northamptonshire': return 'mostly urban'
    elif x == 'East Suffolk': return 'mostly rural'
    elif x == 'North Northamptonshire': return 'mostly urban'
    elif x == 'Somerset West and Taunton': return 'mostly rural'
    elif x == 'West Suffolk': return 'mostly rural'
    elif x == 'Folkestone and Hythe': return 'mostly rural'
    else: 
        try: 
            return la_urban_rural[la_urban_rural['localauth_name'] == x]['class'].values[0]
        except: 
            return np.nan

In [ ]:
def clean_dat(df): 
    df = df[['generalElectionVote', 'p_ethnicityW19', 'p_religionW19', 'p_sexuality', 'p_housing', 'p_work_stat',
             'gender', 'age', 'p_marital', 'p_parent', 'p_education', 'p_gross_household', 
             'oslaua', 'wt']]
    
    df['vote'] = df['generalElectionVote'].map(get_vote)
    df['ethnicity'] = df['p_ethnicity'].map(get_ethncity)
    df['religion'] = df['p_religion'].map(get_religion)
    df['sexuality'] = df['p_sexuality'].map(get_sexuality)
    df['housing'] = df['p_housing'].map(get_housing)
    df['work'] = df['p_work_stat'].map(get_work)
    df['age_cat'] = df['age'].map(get_age_cat)
    df['married'] = df['p_marital'].map(get_married)
    df['children'] = df['p_parent'].map(get_children)
    df['education'] = df['p_education'].map(get_education)
    df['income'] = df['p_gross_household'].map(get_income)
    df['region'] = df['oslaua'].map(get_region)
    df['rural_urban'] = df['oslaua'].map(get_urban_rural)
    
    clean_df = df[['vote', 'ethnicity', 'religion', 'gender', 'housing', 'work', 'sexuality', 'age_cat', 'married', 'children', 'education', 'income', 'region', 'rural_urban', 'wt']].dropna(subset='vote', axis=0)

    return clean_df

In [ ]:
clean_df = clean_dat(pd.read_csv(data_dir / 'british_voter.csv'))